In [3]:
import csv
import pandas as pd
import numpy as np

In [128]:
class Get_data():
#     test_year = None
#     test_term = None
    def __init__(self,year,term,file_name,is_test=False,test_year=None,test_term=None):
        self.df=pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/1.학생 정보.csv',encoding='EUC_KR')
        self.osl = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/2.휴학 및 제적 시간.csv",encoding='EUC_KR')
        self.professor_talk = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/3.교수 상담.csv",encoding='EUC_KR')
        self.point = pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/6.학점 및 평점.csv',encoding='EUC_KR')
        self.late_leave = pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/8.조퇴 및 지각 정보.csv',encoding='EUC_KR')
        self.average_grade = pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/7.교과목별성적.csv',encoding='EUC_KR').fillna(0)
        self.semester = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/9.등록 정보.csv",encoding='EUC_KR')
        self.money = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/10.장학금 정보.csv",encoding='EUC_KR')
        # 我们需要统计的数据截止日期
        self.year = year
        self.term = term
        self.file_name = file_name
        
        self.is_test = is_test
        self.test_year = test_year
        self.test_term = test_term
        if term[0] == "1":
            self.date = 3
        else:
            self.date = 7
        self.true_csv = []
        
        # 총 휴학 학기 계산
        self.xueqi = [[2010, '1학기'], [2010, '2학기'], [2011, '1학기'], [2011, '2학기'], [2012, '1학기'],
                     [2012, '2학기'], [2013, '1학기'], [2013, '2학기'], [2014, '1학기'], [2014, '2학기'],
                     [2015, '1학기'], [2015, '2학기'], [2016, '1학기'], [2016, '2학기'], [2017, '1학기'], 
                     [2017, '2학기'], [2018, '1학기'], [2018, '2학기'], [2019, '1학기'], [2019, '2학기'],
                     [2020, '1학기']]
    
    def get_semester(self,k):
        # 获取一个学生的所有学期，和应该被统计的学期数。
        semester_sum = []
        n = 0
#         all_drop_time = 0
        for n in range(len(k["학년도"])):
            semester_sum.append([k["학년도"].iloc[n],k["학기"].iloc[n]])
        n = 0
        for o in semester_sum:
            uu = pd.to_datetime(str(o[0])+"/"+self.the_date(o[1]))
            if uu >= pd.to_datetime(str(self.year)+"/"+self.the_date(self.term)):
                break
            else:
                n += 1
        # n 就是我们需要的学期数量
        return semester_sum,n

    def get_drop_out_of_school_time(self,semester_sum):
        # 获取休学数
        vv = 0
        all_drop_time = 0
        for vv in self.xueqi[self.xueqi.index(semester_sum[0]):]:
            if vv not in semester_sum:
                # 만약 완전한 학기에 대해 (입학학기부터) 학생들이 지난 학기에 부족함을 발견한다면, +1을 계산한다.
                all_drop_time += 1
            elif vv == semester_sum[-1]:
                break
        return all_drop_time
        
        
        
    def get_point(self,i,semester_number_we_need,per_student_semester):
        # 获取一个学生的学期信息
        every_student = self.point.loc[self.point["학번"] == self.df.at[i,"학번"]]
        # 比对学期信息进行数据提取
        if len(every_student["학번"]) == 0:
            average_apply_point = 0 
            average_getted_point = 0
            average_score = 0
        else:
            apply_point = 0
            getted_point = 0
            score = 0

            # 改进的代码
            every_student["semester"] = pd.to_datetime(every_student["학년도"].map(str) + "/" + every_student["학기"].map(self.the_date))
            a = every_student.loc[every_student["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
            average_apply_point  = a["신청학점"].sum()/semester_number_we_need
            average_getted_point = a["이수학점"].sum()/semester_number_we_need
#             average_score        = score/semester_number_we_need
        return average_apply_point,average_getted_point
    
    def get_scolar(self,i,per_student_semester,semester_number_we_need):
        #평균 장학금 계산
        every_scholarship = self.money.loc[self.money["학번"] == self.df.at[i,"학번"]]
        inschool_scholarship = 0
        outschool_scholarship = 0
        work_scholarship = 0
        grade_scholarship = 0
        arm_scholarship = 0
        poor_scholarship = 0

        every_scholarship["semester"] = pd.to_datetime(every_scholarship["학년도"].map(str) + "/" + every_scholarship["학기"].map(self.the_date))
        a = every_scholarship.loc[every_scholarship["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
        inschool_scholarship = a["입학금장학금비율(총등록금대비)"].sum()+a["수업료장학금비율(총등록금대비)"].sum()
        outschool_scholarship = a["기타장학금비율(총등록금대비)"].sum()
        
        for l in list(a["장학금지급성격"]):
            if l == "근로장학":
                work_scholarship += 1
            elif l == "성적우수":
                grade_scholarship += 1
            elif l == "보훈대상":
                arm_scholarship += 1
            elif l == "가계곤란":
                poor_scholarship += 1
            
        return [inschool_scholarship/semester_number_we_need,outschool_scholarship/semester_number_we_need,
                work_scholarship/semester_number_we_need,grade_scholarship/semester_number_we_need,
                arm_scholarship/semester_number_we_need,poor_scholarship/semester_number_we_need]
    
    def get_change(self,i,per_student_semester):
        # 获取学籍改变，姓名改变等
        every_drop = self.osl.loc[self.osl["학번"] == self.df.at[i,"학번"]]
        all_change = []
#         for p in  per_student_semester:
#             # 确定一个截止抽取数据的时间
#             if p[0] == self.year and p[1] == self.term:
#                 break
#             elif p[0] > self.year:
#                 break
#             try:
#                 for l in list(every_drop.loc[(every_drop["년도"] == p[0] & every_drop["학기"] == int(p[1][0]))]["변동구분"]):
#                     all_change.append(l)
#             except:
#                 continue
        every_drop["semester"] = pd.to_datetime(every_drop["년도"].map(str) + "/" + every_drop["학기"].map(self.the_date))
        a = every_drop.loc[every_drop["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
        all_change = list(a["변동구분"])
        
        cha_special = 0
        change_name = 0
        change_major = 0
        if "폐과로인한전과" in all_change:
            cha_special = 1
            change_major = 0
        if "개명" in all_change:
            change_name = 1
        if "폐과로인한전과" not in all_change and "전과" in all_change:
            change_major = 1
        out_school = ["자퇴제적","미등록제적","미복학제적","미수강제적","학사경고제적","사망제적",
              "(중도수료)제적","유급제적","퇴학제적"]
        # 정보 변경을 중단했는지 확인
        label = 0
        is_dropout = set(all_change) & set(out_school)
        if is_dropout == set():
            label = 0
        else:
            label = 1
    
        return [all_change,cha_special,change_major,change_name,label]
    
    def get_talk_number(self,i,semester_number_we_need):
        # 获取教授商谈次数
        every_student_talk = self.professor_talk.loc[self.professor_talk["학번"] == self.df.at[i,"학번"]]
        every_student_talk["semester"] = pd.to_datetime(every_student_talk["상당일자"])
        
        # 改进的代码
        a = every_student_talk.loc[every_student_talk["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
        talk_number_average = len(a["교수직번"])/semester_number_we_need
        return talk_number_average
    
    def get_evalution_GPA(self,i,semester_number_we_need):
        every_student_grade = self.average_grade.loc[self.average_grade["학번"] == self.df.at[i,"학번"]]
        # 创建时间列
        every_student_grade["semester"] = pd.to_datetime(every_student_grade["년도"].map(str) + "/" + every_student_grade["학기"].map(self.the_date))
        aa = every_student_grade.loc[every_student_grade["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
        if len(aa["강의평가"]) == 0:
            return [0,0]
        evaluation_list = list(aa["강의평가"])
        average_evalution = 0
        # 평균 평점
        the_point_list = list(aa["학점수"])
        the_grade_list = list(aa["평점"])
        grade_sum = 0
        for ll in range(len(aa["학번"])):
            grade_sum += the_grade_list[ll] * the_point_list[ll]
        if aa["학점수"].sum() == 0:
            true_grade = 0
        else:
            true_grade = grade_sum/aa["학점수"].sum()
        return [true_grade,sum(evaluation_list)/len(aa["강의평가"])]
    
    def get_late(self,i,semester_number_we_need):
        student_not_in_time = self.late_leave.loc[self.late_leave["학번"] == self.df.at[i,"학번"]]
        # 创建时间列
        student_not_in_time["semester"] = pd.to_datetime(student_not_in_time["연도"].map(str) + "/" + student_not_in_time["학기"].map(self.the_date))
        
        aa = student_not_in_time.loc[student_not_in_time["semester"] < pd.to_datetime(str(self.year) + "/" + str(self.date))]
        not_come = aa["결석"].sum()/semester_number_we_need
        not_come_reason = aa["공결"].sum()/semester_number_we_need
        be_late = aa["지각"].sum()/semester_number_we_need
        
        return [not_come,not_come_reason,be_late]
        
    def the_date(self,da):
        if da == "1학기":
            return "3"
        else:
            return "7"
        
        
        
    def give_me_data(self):
        for i in range(self.df.shape[0]):
            k = self.semester.loc[self.semester["학번"] == self.df.at[i,"학번"]].sort_values(["학년도","학기"])
            # 학생들이 학기 정보를 가지고 있는지 확인
            try:
                min(k["학년도"])
            except:
                continue
            else:
                if list(k["학년도"])[0] < 2017:
                    continue
                elif len(k["학기"]) == 0:
                    continue
                """
                学期信息
                """
                per_student_semester,semester_number_we_need = self.get_semester(k)
                if semester_number_we_need == 0:
                    continue
                if self.is_test:
                    if [self.test_year,self.test_term] not in per_student_semester:
                        continue
                
                """
                学分信息
                """
                average_apply_point,average_getted_point = self.get_point(i,semester_number_we_need,per_student_semester)
                """
                奖学金信息
                """
                scolar_info = self.get_scolar(i,per_student_semester,semester_number_we_need)
                """
                总休学数
                """
                drop_out_time = self.get_drop_out_of_school_time(per_student_semester)
                """
                学籍变动
                """
                change_list = self.get_change(i,per_student_semester)
                """
                教授商谈次数
                """
                talk_number_average = self.get_talk_number(i,semester_number_we_need)
                """
                讲义评价，GPA
                """
                evalution_GPA = self.get_evalution_GPA(i,semester_number_we_need)
                """
                缺席和迟到
                """
                late_list = self.get_late(i,semester_number_we_need)
                """
                学年数
                """
                study_year = semester_number_we_need//2+semester_number_we_need%2
                """
                整理单个学生的所有数据
                """
                cache = [self.df.at[i,"학번"],self.df.at[i,"대학"],self.df.at[i,"학과"],self.df.at[i,"성별"],self.df.at[i,"출생년도"],
                         self.df.at[i,"거주지역"],self.df.at[i,"입시총점"],self.df.at[i,"입학모집구분"],self.df.at[i,"고교유형"],
                         self.df.at[i,"예비역여부"],average_apply_point,average_getted_point,average_apply_point-average_getted_point,
                         scolar_info[0],scolar_info[1],scolar_info[2],scolar_info[3],scolar_info[4],scolar_info[5],
                         drop_out_time,
                         change_list[0],change_list[1],change_list[2],change_list[3],change_list[4],
                         talk_number_average,
                         evalution_GPA[0],evalution_GPA[1],
                         late_list[0],late_list[1],late_list[2],
                         study_year,
                         per_student_semester,semester_number_we_need]
                self.true_csv.append(cache)
    def to_csv(self):
        pd.set_option('display.max_columns', None)
#         pd.set_option('display.max_rows', None)
        fina = pd.DataFrame(self.true_csv,columns=["ID","대학","학과","성별","출생년도","거주지역","입시총점"
                                              ,"입학모집구분","고교유형","예비역여부",
                                              "평균 신청 학점","평균 이수 학점","평균 신청 이수 학점 차이",
                                              "평균 교내 장학금 비율","평균 교외 장학금 비율","근로 장학금 취득 횟수","성적 장학금 취득 횟수","공훈 장학금 취득 횟수","가계곤란 장학금 취득 횟수",
                                              "총휴학 학기","모든 학사 변동","폐과로인한전과유무","개명여부","전과유무","제적 여부",
                                              "평균 교수와 상담 횟수",
                                              "GPA","평균 강의 평가",
                                              "평균 결석수","평균 공결수","평균 지각수",
                                              "다닌 학년",
                                              "총 등록 학기","계산한 학기수",
                                              ])
        
        fina.to_csv(self.file_name+".csv",encoding="EUC_KR")                              
        return fina                    
        

In [129]:
a = Get_data(2019,"2학기","test_data_0228",is_test=True,test_year=2019,test_term="1학기")

D:\anaconda\envs\work\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
D:\anaconda\envs\work\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (8,17,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [130]:
a.give_me_data()

D:\anaconda\envs\work\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda\envs\work\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda\envs\work\lib\site-packages\ipykernel_launcher.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [85]:
a.head()

AttributeError: 'Get_data' object has no attribute 'head'

In [38]:
df=pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/1.학생 정보.csv',encoding='EUC_KR')
osl = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/2.휴학 및 제적 시간.csv",encoding='EUC_KR')
professor_talk = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/3.교수 상담.csv",encoding='EUC_KR')
point = pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/6.학점 및 평점.csv',encoding='EUC_KR')
average_grade = pd.read_csv('E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/7.교과목별성적.csv',encoding='EUC_KR')
semester = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/9.등록 정보.csv",encoding='EUC_KR')
money = pd.read_csv("E:/python doc/备份/data_study/jupyter/学生数据解析111/学生数据解析/정리 파일/원자료/20191028_data/10.장학금 정보.csv",encoding='EUC_KR')

D:\anaconda\envs\work\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\anaconda\envs\work\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (8,17,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
every_student_grade = average_grade.loc[average_grade["학번"] == "7add0bce10b91d072a72a078a60b9e7854a24d0be594afa05ba3421a021e0a5c"]

In [50]:
every_student_grade

,학번,년도,학기,교과목번호,분반,교과목명,이수구분,수강인원,개설학과명,학점수,...,평점,강의평가,삭제구분,재수강년도1,재수강학기1,재수강교과번호1,재수강년도2,재수강학기2,재수강교과번호2,semester
13,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,100001,101.0,실용영어회화,공통교양,30,영어영문학과,2.0,...,4.5,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
4833,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,100003,104.0,정보처리개론,공통교양,50,경영정보학과,2.0,...,3.0,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
6904,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,100004,108.0,신입생강좌,공통교양,18,사학과,1.0,...,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
14038,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,100006,1.0,실용한자,공통교양,718,교양교육원,1.0,...,4.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
19862,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,101303,10.0,한국사회의이해,핵심교양,141,사회학과,2.0,...,3.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
22106,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,101362,10.0,사랑의기법을통한교육,핵심교양,61,교육학과,2.0,...,2.5,1.0,학점포기,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
27375,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,101701,10.0,실용영어연습,핵심교양,70,영어영문학과,2.0,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
28479,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,101731,10.0,영미문학과영화,핵심교양,111,영어영문학과,2.0,...,2.5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
29017,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,101742,10.0,프랑스문화의이해,핵심교양,115,프랑스문화학과,2.0,...,3.5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01
40664,7add0bce10b91d072a72a078a60b9e7854a24d0be594af...,2010,1학기,970013,1.0,서양사영문강독,학문기초교양,57,사학과,3.0,...,2.5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01


In [49]:
# 创建时间列
def the_date(da):
    if da == "1학기":
        return "3"
    else:
        return "7"
every_student_grade["semester"] = pd.to_datetime(every_student_grade["년도"].map(str) + "/" + every_student_grade["학기"].map(the_date))

D:\anaconda\envs\work\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [32]:
every_student_talk["ll"] = every_student_talk["교수직번"].map(str) +"/"+ every_student_talk["교수성명"]

D:\anaconda\envs\work\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
every_student_talk

In [34]:
def the_date(da):
    if da == 811150:
        return 3
    else:
        return 7
every_student_talk["교수직번"].map(the_date)

16       3
7796     3
29020    7
60434    7
Name: 교수직번, dtype: int64

In [23]:
every_student_talk.loc[every_student_talk["semester"] < pd.to_datetime("2012/07")]

,학번,교수직번,교수성명,상당일자,semester
16,290fdb5720ebe7cc8a5d0ccbdc0b203aae5c3ed66143a1...,811150,김은이,2010-03-11,2010-03-11
7796,290fdb5720ebe7cc8a5d0ccbdc0b203aae5c3ed66143a1...,811150,김은이,2010-12-28,2010-12-28
29020,290fdb5720ebe7cc8a5d0ccbdc0b203aae5c3ed66143a1...,121850,정찬식,2012-06-06,2012-06-06
